In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow --quiet

In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.spatial.distance import cosine
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GlobalMaxPool1D, Dense
from tensorflow.keras.models import Sequential

In [15]:
#Loading the dataset
df = pd.read_csv("PLCdata.csv",index_col=[0])
#combining the fault, model and PLC columns into 1 column fault_model
df['Fault_Model'] = df['Fault'] + " " + df['Model']+ " " + df["PLC"]

df.drop(['Fault','Model','PLC'], inplace=True, axis=1)

In [16]:
df.head()

,Diagnostic,Corrective Action,Fault_Model
0,broken or cracked glass electrode,replace electrode if cracked,cracked glass failure model e emerson
1,broken or cracked glass electrode,check wiring for short,cracked glass failure model e emerson
2,reference electrode poisoned,replace reference electorode,zero offset error model e emerson
3,coated reference electrode,clean electrode as instructed in sensor manual,high reference imped model e emerson
4,sensor out of process,clean electrode as instructed in sensor manual,high reference imped model e emerson


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1398 entries, 0 to 1397
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Diagnostic         1398 non-null   object
 1   Corrective Action  1397 non-null   object
 2   Fault_Model        1398 non-null   object
dtypes: object(3)
memory usage: 43.7+ KB


In [25]:
# Build the model
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
    model.add(tf.keras.layers.GlobalMaxPool1D())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model
model= create_model()


# Example query sentence
query = "high reference imped model e emerson"

df=df
# Tokenize the  and the query
tokenizer = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')
tokenizer.fit_on_texts(df['Fault_Model'].append(pd.Series(query)))
encoded_summaries = tokenizer.texts_to_sequences(df['Fault_Model'])
encoded_query = tokenizer.texts_to_sequences([query])

# Pad the encoded summaries and the encoded query to the maximum length
max_length = max([len(seq) for seq in encoded_summaries + encoded_query])
padded_summaries = pad_sequences(encoded_summaries, maxlen=max_length, padding='post')
padded_query = pad_sequences(encoded_query, maxlen=max_length, padding='post')

model.fit(padded_summaries,padded_query, epochs=5, batch_size=32, validation_split=0.1, verbose=1)


# Get the embeddings of the summaries and the query
summaries_embeddings = model.predict(padded_summaries)
query_embedding = model.predict(padded_query)

# Calculate the cosine similarity between the query embedding and each of the summary embeddings
similarities = [1 - cosine(query_embedding[0], summary_embedding[0]) for summary_embedding in summaries_embeddings]

# Get the index of the closest summary to the query
closest_summary_index = np.argmax(similarities)

# Get the closest title and summary
diagnostic = df['Diagnostic'][closest_summary_index]
correctiveaction = df['Corrective Action'][closest_summary_index]
print('Diagnostic:',diagnostic)
print('Corrective Action:',correctiveaction)

ValueError: ignored

In [ ]:
!pip install streamlit --quiet
! pip install pyngrok --quiet

In [ ]:
%%writefile app.py
import streamlit as st
st.write("# PLC Fault Prediction")
st.text_input("Enter the PLC:")
st.text_input("Enter the Model:")
st.text_input("Enter the paper Fault:")

Overwriting app.py


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2LArVCZnMmuPMV0uYjEx4zAOSpx_6AiMAuvnRpVMQxFzSkvaE") 

!nohup streamlit run app.py --server.port 80 & --quiet
url = ngrok.connect(port = '80')
print(url)

/bin/bash: --quiet: command not found
nohup: appending output to 'nohup.out'
NgrokTunnel: "http://6b6c-34-83-144-170.ngrok.io" -> "http://localhost:80"


In [ ]:
ngrok.kill()